In [1]:
import os
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\text_summarization\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\text_summarization'

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_dirs

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    data_path:Path
    model_ckpt:Path
    num_train_epochs:int
    warmup_steps:int
    per_device_train_batch_size:int
    per_device_eval_batch_size:int
    weight_decay:float
    logging_steps:int
    save_steps:float
    gradient_accumulation_steps:int
    do_eval:bool


In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_dirs([self.config.artifacts_root])
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        
        create_dirs([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            do_eval=params.do_eval
        )
        return model_trainer_config

In [7]:
import torch
from datasets import load_dataset,load_from_disk
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer,DataCollatorForSeq2Seq,Trainer,TrainingArguments

d:\Jeevan\VSCode\Text_summarization\text_summarization\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-08 21:01:47,658 : INFO : config : PyTorch version 2.6.0 available. ]


In [8]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config
    
    def train(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        dataset_samsum_pt=load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=500,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            weight_decay=0.01,
            logging_steps=10,
            save_steps=int(1e6),
            gradient_accumulation_steps=16,
            remove_unused_columns=True,
            do_eval=True
        )

        trainer = Trainer(model=model_pegasus,
                  args=trainer_args,
                  tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"]
                  )
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [ ]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except ValueError as e:
    raise e